In [2]:
import os

import pymongo
from bson.json_util import dumps, loads

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

import pandas as pd

import re
from pprint import pprint
from datetime import datetime
import time

import sqlalchemy
from sqlalchemy import create_engine, MetaData, inspect, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, Date, DateTime, Boolean, BigInteger
from sqlalchemy.orm import sessionmaker, relationship


import sys
sys.path.append('C:\\Users\\soria\\Documents\\WashUDataDocuments\\HwActivites\\YouTube_Project\\')
from config import YOUTUBE_DATA_API_KEY
import pandas as pd
import json
import os
from pprint import pprint

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

Connect to mongoDb client and create database name

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.youtube
#   collection.insert_one(test_programs)
#   collection = db[program_name]
#   collection.update_one({}, {'$set': kdhx_dict}, upsert=True)


Connect to local mySql, create engine, declare base

In [4]:
rds_connection_string = "root:Password@127.0.0.1"
engine = create_engine(f'mysql://{rds_connection_string}')
engine.execute('CREATE DATABASE IF NOT EXISTS youtube_db')
engine.execute('USE youtube_db')
Base = declarative_base()


base class for region_codes

In [5]:
class Region_Codes(Base):
    __tablename__ = 'region_codes'
    id = Column(Integer, primary_key=True)
    region = Column(String(256))
    region_code = Column(String(256))
    cat_ids = relationship("Category_Ids", backref="cat_region_codes")
    
    def __init__(self, region, region_code):
        self.region = region
        self.region_code = region_code
        
        
    def __repr__(self):
        return f"id={self.id}, region={self.region}, region_code={self.region_code}"
    
#     Region_Codes().cat_ids.append(Category_Ids())

base class for category_ids

In [6]:
class Category_Ids(Base):
    __tablename__ = 'category_ids'
    id = Column(Integer, primary_key=True)
    region_codes_table_id = Column(Integer, ForeignKey('region_codes.id'))
    category_title = Column(String(256))
    category_id = Column(String(256))
    category_is_assignable = Column(Boolean)
    region_code = Column(String(256))
    vids = relationship("Videos", backref="vid_category_ids")
    
    def __init__(self, category_title, category_id, category_is_assignable, region_code):
        self.category_title = category_title
        self.category_id = category_id
        self.category_is_assignable = category_is_assignable
        self.region_code = region_code
        
    def __repr__(self):
        return f"id={self.id}, category_title={self.category_title}, category_id={self.category_id}"
  
    #     Category_Ids().vids.append(Video())

base class for topic_ids

In [7]:
class Topic_Ids(Base):
    __tablename__ = 'topic_ids'
    id = Column(Integer, primary_key=True)
    topic_name = Column(String(256))
    topic_id = Column(String(256))
    topic_heading = Column(String(256))
    channs = relationship("Channel", backref="chann_topic_ids")
    vids = relationship("Videos", backref="vid_topic_ids")
    
    def __init__(self, topic_name, topic_id, topic_heading):
        self.topic_name = topic_name
        self.topic_id = topic_id
        self.topic_heading = topic_heading
        
    def __repr__(self):
        return f"id={self.id}, topic_name={self.topic_name}, topic_id={self.topic_id}, topic_heading={self.topic_heading}"
    
#     Topic_Ids().channs.append(Channel())
#     Topic_Ids().vids.append(Video())


base class for channels

In [8]:
class Channel(Base):
    __tablename__ = 'channels'
    id = Column(Integer, primary_key=True)
    topic_ids_table_id = Column(Integer, ForeignKey('topic_ids.id'))
    channel_title = Column(String(256))
    channel_url = Column(String(256))
    channel_id = Column(String(256))
    channel_keywords = Column(String(256))
    channel_description = Column(String(1024))
    subscribers = Column(BigInteger)
    video_count = Column(Integer)
    view_count = Column(BigInteger)
    country = Column(String(256))
    join_date = Column(Date)
    topic_ids = Column(String(256))
    cc_category = Column(String(256))
    thumbnail_url = Column(String(256))
    vids = relationship("Videos", backref="vid_channels")
    
    def __init__(self, topic_ids_table_id, channel_title, channel_url, channel_id, channel_keywords, channel_description,\
                 subscribers, video_count, view_count, country, join_date, topic_ids, cc_category, thumbnail_url):
        self.topic_ids_table_id = topic_ids_table_id
        self.channel_title = channel_title
        self.channel_url = channel_url
        self.channel_id = channel_id
        self.subscribers = subscribers
        self.video_count = video_count
        self.view_count = view_count
        self.country = country
        self.join_date = join_date
        self.join_dto = join_dto
        self.topic_ids = topic_ids
        self.cc_category = cc_category
        self.thumbnail_url = thumbnail_url
        
        
    def __repr__(self):
        return f"id={self.id}, channel_title={self.channel_title}, url={self.channel_url}"
    
    #     Channel().vids.append(Video())

base class for videos

In [9]:
class Videos(Base):
    __tablename__ = 'videos'
    id = Column(Integer, primary_key=True)
    channels_table_id = Column(Integer, ForeignKey('channels.id'))
    topic_ids_table_id = Column(Integer, ForeignKey('topic_ids.id'))
    category_ids_table_id = Column(Integer, ForeignKey('category_ids.id'))
    video_title = Column(String(256))
    video_description = Column(String(1024))
    view_count = Column(BigInteger)
    duration = Column(Integer) # https://stackoverflow.com/questions/15596753/how-do-i-get-video-durations-with-youtube-api-version-3
    like_count = Column(Integer)
    dislike_count = Column(Integer)
    favorite_count = Column(Integer)
    comment_count = Column(Integer)
    published_at = Column(DateTime)
    published_at_dto = Column(DateTime)
    tags = Column(String(256))
    dimension = Column(String(256))
    definition = Column(String(256))
    caption = Column(Boolean)
    licensedContent = Column(Boolean)
    license = Column(String(256))
    
    
    def __init__(self, channels_table_id, topic_ids_table_id, category_ids_table_id, video_title, video_description,\
                 view_count, duration, like_count, dislike_count, favorite_count, comment_count, published_at,\
                 published_at_dto, tags, dimension, definition, caption, licensedContent, license):
        self.channels_table_id = channels_table_id
        self.topic_ids_table_id = topic_ids_table_id
        self.category_ids_table_id = category_ids_table_id
        self.video_title = video_title
        self.view_count = view_count
        self.duration = duration
        self.like_count = like_count
        self.dislike_count = dislike_count
        self.favorite_count = favorite_count
        self.comment_count = comment_count
        self.published_at = published_at
        self.published_at_dto = published_at_dto
        self.tags = tags
        self.dimension = topic_ids
        self.definition = definition
        self.caption = caption
        self.licensedContent = licensedContent
        self.license = license
        
        
    def __repr__(self):
        return f"id={self.id}, video_title={self.video_title}, view_count={self.view_count}"
    


In [10]:
Session = sessionmaker(bind=engine)
sqlDbSession = Session()

Create a "Metadata" Layer That Abstracts our SQL Database

In [11]:


Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

point splinter to chromedriver, assign browser object to variable

In [12]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)


declare url to gist site hosting topic id list, visit and wait for necessary element to load

In [13]:
url = "https://gist.github.com/stpe/2951130dfc8f1d0d1a2ad736bef3b703"
driver = webdriver.Chrome()
driver.get(url)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.element_to_be_clickable((By.ID,'user-content-other-topics')))


In [14]:
client.list_database_names()

['ClassDB',
 'admin',
 'config',
 'craigslist_app',
 'craigslist_db',
 'fruits_db',
 'kdhx',
 'kdhx_shows',
 'local',
 'mars_app',
 'nhl_db',
 'store_inventory',
 'team_db',
 'travel_db',
 'weather_app']

In [15]:
mongoDb.list_collection_names()

[]

In [16]:
collection = mongoDb.topicId_gist_html
if mongoDb.topicId_gist_html.find_one():
    html = mongoDb.topicId_gist_html.find_one()['channel_crawler_html']
else:
    html = driver.page_source
topic_id_html_dict = {'topicId_gist_url': driver.current_url, 'channel_crawler_html':html}
collection.update_one({}, {'$set': topic_id_html_dict}, upsert=True)
topic_soup = bs(html, 'html.parser')
heading_list = []
topic_headings = topic_soup.find_all('h2')
for heading in topic_headings:
    heading_list.append(heading.text)
topics_dict = {}
topic_dict_list = []
topics = topic_soup.find_all('li')
for index, topic in enumerate(topics[4:20]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[20:31]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[31:45]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[45:50]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[50:61]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[61:63]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()

In [17]:
topic_results = sqlDbSession.query(Topic_Ids.topic_name, Topic_Ids.topic_id, Topic_Ids.topic_heading)\
                    .all()
topic_name = [result[0] for result in topic_results]
topic_ids = [result[1] for result in topic_results]
topic_headings = [result[2] for result in topic_results]

In [18]:
topic_df = pd.DataFrame(topic_results, columns=['topic_name', 'topic_ids', 'topic_headings'])
# topic_df.set_index('topic_name', inplace=True, )
topic_df.head()
topic_df.to_csv('./../../../youtube_project_data/youtube_topic_ids.csv')

In [19]:
sqlDbSession.close()

In [20]:
rds_connection_string = "root:Password@127.0.0.1"
engine = create_engine(f'mysql://{rds_connection_string}')
# engine.execute('CREATE DATABASE IF NOT EXISTS youtube_db')
engine.execute('USE youtube_db')
topic_stmt = sqlDbSession.query(Topic_Ids).statement
topic_df2 = pd.read_sql_query(topic_stmt, con=engine)
topic_df2.head()

,id,topic_name,topic_id,topic_heading
0,1,Music,/m/04rlf,Music topics
1,2,Children's music,/m/05fw6t,Music topics
2,3,Christian music,/m/02mscn,Music topics
3,4,Classical music,/m/0ggq0m,Music topics
4,5,Country,/m/01lyv,Music topics


In [21]:
Session = sessionmaker(bind=engine)
sqlDbSession = Session()

In [22]:
DEVELOPER_KEY = YOUTUBE_DATA_API_KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


In [23]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

list of region codes

In [24]:
mongoDb.regionCode_ytApi_resonse.find_one()

In [25]:
if mongoDb.topicId_gist_html.find_one():
    html = mongoDb.topicId_gist_html.find_one()['channel_crawler_html']

In [26]:
collection = mongoDb.regionCode_ytApi_response
if mongoDb.regionCode_ytApi_resonse.find_one():
    region_code_search_response = mongoDb.regionCode_ytApi_response.find_one()['region_code_ytApi_response']
else:
    region_code_search_response = youtube.i18nRegions().list(
    part="snippet,id"
        
    ).execute()
region_code_response_dict = {'region_code_ytApi_response': region_code_search_response}
collection.update_one({}, {'$set': region_code_response_dict}, upsert=True)
region_code_dict_list = []
region_codes_dict = {"region_codes": []}
for item in region_code_search_response['items']:
    region = item['snippet']['name']
    region_code = item['snippet']['gl']
    region_codes_dict['region_codes'].append({'region': region, 'region_code': region_code})
    collection = mongoDb.youtube_regionCodes
    collection.update_one({}, {'$set': region_codes_dict}, upsert=True)
    region_code_dict_list.append({'region': region, 'region_code': region_code})
    sqlDbSession.add(Region_Codes(region, region_code))
    sqlDbSession.commit()
    


In [27]:
region_code_stmt = sqlDbSession.query(Region_Codes).statement
region_code_df = pd.read_sql_query(region_code_stmt, con=engine)
region_code_df.head()

,id,region,region_code
0,1,Algeria,DZ
1,2,Argentina,AR
2,3,Australia,AU
3,4,Austria,AT
4,5,Azerbaijan,AZ


In [28]:
for key in region_code_dict_list[1]:
    print(key)

region
region_code


In [29]:
region_code_dict_list[0][key]

'DZ'

In [30]:
for i, item in enumerate(region_code_dict_list):
    item.setdefault('categories', {})
    item['categories'] = i
region_code_dict_list

[{'region': 'Algeria', 'region_code': 'DZ', 'categories': 0},
 {'region': 'Argentina', 'region_code': 'AR', 'categories': 1},
 {'region': 'Australia', 'region_code': 'AU', 'categories': 2},
 {'region': 'Austria', 'region_code': 'AT', 'categories': 3},
 {'region': 'Azerbaijan', 'region_code': 'AZ', 'categories': 4},
 {'region': 'Bahrain', 'region_code': 'BH', 'categories': 5},
 {'region': 'Belarus', 'region_code': 'BY', 'categories': 6},
 {'region': 'Belgium', 'region_code': 'BE', 'categories': 7},
 {'region': 'Bolivia', 'region_code': 'BO', 'categories': 8},
 {'region': 'Bosnia and Herzegovina', 'region_code': 'BA', 'categories': 9},
 {'region': 'Brazil', 'region_code': 'BR', 'categories': 10},
 {'region': 'Bulgaria', 'region_code': 'BG', 'categories': 11},
 {'region': 'Canada', 'region_code': 'CA', 'categories': 12},
 {'region': 'Chile', 'region_code': 'CL', 'categories': 13},
 {'region': 'Colombia', 'region_code': 'CO', 'categories': 14},
 {'region': 'Costa Rica', 'region_code': 'CR'

category_Id search

In [31]:
all_region_category_list = []
video_category_response_dict = {}
region_categories_dict = {}
for region in region_code_dict_list:
    region_code = region['region_code']
    video_category_search_response = youtube.videoCategories().list(
        regionCode=region_code,
        part="snippet,id"
    ).execute()
    video_category_response_dict.setdefault(region_code, {})
    video_category_response_dict[region_code] = video_category_search_response
    collection = mongoDb.videoCategory_ytApi_response
#     collection.insert_one({region_code: video_category_search_response})
    collection.update_one({}, {'$set': video_category_response_dict}, upsert=True)
    region_categories_dict.setdefault(region_code, [])
    region_category_list = []
    for item in video_category_search_response['items']:
        category_title = item['snippet']['title']
        category_id = item['id']
        category_is_assignable = item['snippet']['assignable']
        category_id_dict = {'category_title': category_title, 'category_id': category_id,\
                            'category_is_assignable': category_is_assignable, 'region_code': region_code}
        region_category_list.append(category_id_dict)
        region_categories_dict[region_code].append(category_id_dict)
        sqlDbSession.add(Category_Ids(category_title, category_id, category_is_assignable, region_code)) 
        sqlDbSession.commit()
        region_categories_dict[region_code].append(category_id_dict)
    collection = mongoDb.youtube_videoCategories
    collection.update_one({}, {'$set': region_categories_dict}, upsert=True)
    region['categories'] = region_category_list
    all_region_category_list.append(region_category_list)


In [33]:
collection = mongoDb.videoCategory_ytApi_response
if mongoDb.videoCategory_ytApi_response.find_one():
#     video_category_search_response = mongoDb.video_category_search_response.find_one()['region_code_ytApi_response']
    all_region_category_list = []
    video_category_response_dict = {}
    region_categories_dict = {}
    for region in region_code_dict_list:
        region_code = region['region_code']
        collection = mongoDb.videoCategory_ytApi_response
        video_category_search_response = loads(dumps(collection.find({region_code : {'$exists':True}})))
        for response in video_category_search_response:
            region_categories_dict.setdefault(region_code, [])
            region_category_list = []
            for item in response[region_code]['items']:
                category_title = item['snippet']['title']
                category_id = item['id']
                category_is_assignable = item['snippet']['assignable']
                category_id_dict = {'category_title': category_title, 'category_id': category_id,\
                                    'category_is_assignable': category_is_assignable, 'region_code': region_code}
                region_category_list.append(category_id_dict)
                region_categories_dict[region_code].append(category_id_dict)
                sqlDbSession.add(Category_Ids(category_title, category_id, category_is_assignable, region_code)) 
                sqlDbSession.commit()
                region_categories_dict[region_code].append(category_id_dict)
            collection = mongoDb.youtube_videoCategories
            collection.update_one({}, {'$set': region_categories_dict}, upsert=True)
            region['categories'] = region_category_list
            all_region_category_list.append(region_category_list)
else:
    all_region_category_list = []
    video_category_response_dict = {}
    region_categories_dict = {}
    for region in region_code_dict_list:
        region_code = region['region_code']
        video_category_search_response = youtube.videoCategories().list(
            regionCode=region_code,
            part="snippet,id"
        ).execute()
        video_category_response_dict.setdefault(region_code, {})
        video_category_response_dict[region_code] = video_category_search_response
        collection = mongoDb.videoCategory_ytApi_response
    #     collection.insert_one({region_code: video_category_search_response})
        collection.update_one({}, {'$set': video_category_response_dict}, upsert=True)
        region_categories_dict.setdefault(region_code, [])
        region_category_list = []
        for item in video_category_search_response['items']:
            category_title = item['snippet']['title']
            category_id = item['id']
            category_is_assignable = item['snippet']['assignable']
            category_id_dict = {'category_title': category_title, 'category_id': category_id,\
                                'category_is_assignable': category_is_assignable, 'region_code': region_code}
            region_category_list.append(category_id_dict)
            region_categories_dict[region_code].append(category_id_dict)
            sqlDbSession.add(Category_Ids(category_title, category_id, category_is_assignable, region_code)) 
            sqlDbSession.commit()
            region_categories_dict[region_code].append(category_id_dict)
        collection = mongoDb.youtube_videoCategories
        collection.update_one({}, {'$set': region_categories_dict}, upsert=True)
        region['categories'] = region_category_list
        all_region_category_list.append(region_category_list)


In [ ]:
for item in video_category_search_response['items']:
    print(item)

In [ ]:
code = "DZ"
collection = mongoDb.videoCategory_ytApi_response
this = collection.find({code : {'$exists':True}})
dump = dumps(this)
load = loads(dump)
# load[0]['DZ']['items']
load

In [ ]:
categoryKeys = []
for k, v in video_category_search_response['items'][0]['snippet'].items():
    categoryKeys.append(k)
categoryKeys = categoryKeys
categoryKeys

In [ ]:

for item in video_category_search_response['items']:
    print(f"id: {item['id']}")
    print(f"  title: {item['snippet']['title']}")
    print(f"  assignable: {item['snippet']['assignable']}")
    print("========")
    

In [ ]:
sqlDbSession.close()